In [112]:
import datetime as dtm
import requests
import pandas as pd
import time
import os
import numpy as np
import scipy as sp
import pickle
from xgboost import XGBRegressor, XGBClassifier
from xgboost import plot_importance
import matplotlib.pyplot as plt
import src.data_retriever as intrinio
import src.pso as pso
import src.trading_bot as bot
import random
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
%matplotlib inline

In [113]:
random.seed(42)
datefrom = dtm.datetime(2010, 1, 1)
dateto = dtm.datetime(2019, 3, 1)

In [114]:
#DOW 30
# stock_names = ['AAPL', 'CSCO', 'WMT', 'MCD', 'GS', 'AXP', 'BA', 'KO', 'V', 'WMT', 'UNH']
stock_names = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'XOM', 'GS',
               'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
               'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']
# stock_names = ['AAPL']

In [115]:
stocks_filename = 'stocks_dow30_full.csv'
if os.path.isfile(stocks_filename):
    stocks = pd.read_csv(stocks_filename)
    stocks['date'] = pd.to_datetime(stocks['date'], format="%Y-%m-%d")
else:
    stocks = intrinio.get_stocks_data(stock_names, datefrom, dateto)
    stocks.to_csv(stocks_filename, index_label='id', columns=stocks.columns)

### Подготовка к тестированию

In [116]:
train_datefrom = datefrom
train_dateto = dtm.datetime(2017, 1, 1)

valid_datefrom = train_dateto
valid_dateto = dtm.datetime(2018, 1, 1)

test_datefrom = dtm.datetime(2018, 1, 1)
test_dateto = dtm.datetime(2019, 1, 1)

In [117]:
data = stocks.drop(0).reset_index(drop=True)
data['date'] = data['date'].map(lambda x: x.timestamp())

In [118]:
#to_normalize = ['volume_prev1', 'ebitda']
data['date_norm'] = MinMaxScaler().fit_transform(data['date'].as_matrix().reshape(-1, 1))
#data['volume_prev1'] = MinMaxScaler().fit_transform(data['volume_prev1'].as_matrix().reshape(-1, 1))
#data['ebitda'] = MinMaxScaler().fit_transform(data['ebitda'].as_matrix().reshape(-1, 1))
lbl_encoder=LabelEncoder()
data['stock_name'] = lbl_encoder.fit_transform(data['stock_name'].as_matrix().reshape(-1, 1))
# test_data = data[data['stock_name'] == 0]
# data = data[data['stock_name'] != 0]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [119]:
lbl_encoder.classes_

array(['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM',
       'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE',
       'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM'], dtype=object)

In [120]:
lbl_encoder.inverse_transform(np.array([1]))

array(['AXP'], dtype=object)

In [121]:
train_data = data[data['date'] <= train_dateto.timestamp()]
X_train = train_data.drop(['date','close','adj_close', 'volume'],axis=1)
print(len(X_train))
Y_train_regr = train_data['close']
Y_train_classify = train_data['close'] > train_data['open']

49336


In [122]:
valid_data = data[data['date'] > valid_datefrom.timestamp()]
valid_data = valid_data[valid_data['date'] < valid_dateto.timestamp()]
# valid_data = valid_data.query('(stock_name == 0)')
X_valid = valid_data.drop(['date','close','adj_close', 'volume'],axis=1)
print(len(X_valid))
Y_valid_regr = valid_data['close']
Y_valid_classify = valid_data['close'] > valid_data['open']

7028


In [127]:
test_data = data[data['date'] > test_datefrom.timestamp()]
# test_data = test_data[test_data['date'] < test_dateto.timestamp()]
# test_data = test_data[test_data['stock_name'] == lbl_encoder.transform(np.array(['AAPL']))[0]]
# test_data = test_data[test_data['stock_name'] == lbl_encoder.transform(np.array(['UNH']))[0]]
# test_data = test_data[test_data['stock_name'] == lbl_encoder.transform(np.array(['MMM']))[0]]
len(test_data)

7824

In [128]:
test_data.head()

,id,adi,adj_close,adj_close_prev1,adj_open,adx-3-7-adx,adx-3-7-di_neg,adx-3-7-di_pos,adx-7-14-adx,adx-7-14-di_neg,...,rsi-14,rsi-28,rsi-7,sma-14,sma-28,sma-7,stock_name,volume,volume_prev1,date_norm
2012,2013,-1.458311e+07,110.054822,105.831788,107.249310,42.750594,15.221294,36.481842,42.750594,15.221294,...,59.281610,58.813974,60.414261,109.352143,107.049286,108.731429,6,11014303.0,4538403.0,0.873467
2013,2014,-1.500784e+07,110.527330,110.054822,110.438734,43.042990,14.222901,39.291045,43.042990,14.222901,...,66.697693,63.025303,76.422229,109.680000,107.378571,109.084286,6,9237926.0,11014303.0,0.873766
2014,2015,-1.592825e+07,110.478110,110.527330,111.186871,43.314500,13.473545,37.220935,43.314500,13.473545,...,68.162486,63.922663,78.771495,110.022857,107.726071,109.578571,6,7417409.0,9237926.0,0.874065
2015,2016,-1.876549e+07,109.877632,110.478110,110.921086,43.242744,14.239530,35.126070,43.242744,14.239530,...,67.034419,63.356169,75.647426,110.407857,108.075000,110.177143,6,6008285.0,7417409.0,0.874364
2016,2017,-2.237528e+07,108.302608,109.877632,109.159027,41.889237,19.672691,32.298360,41.889237,19.672691,...,59.950755,59.717633,57.889653,110.330000,108.370000,110.530000,6,8052594.0,6008285.0,0.875262


In [125]:
ts = time.time()
class_params = {
  'colsample_bytree': 0.6617438163723828,
  'eta': 0.1447537144916002,
  'max_depth': 32,
  'min_child_weight': 173,
  'n_estimators': 2725,
  'subsample': 0.31509874620111583,
  'seed': 42}
model_class = XGBClassifier(**class_params)

model_class.fit(
        X_train, 
        Y_train_classify, 
        eval_set=[(X_train, Y_train_classify), (X_valid, Y_valid_classify)], 
        verbose=True,         
        eval_metric='auc',
        early_stopping_rounds = 10)
    
time.time() - ts

[0]	validation_0-auc:0.635805	validation_1-auc:0.6199
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.658442	validation_1-auc:0.64673
[2]	validation_0-auc:0.670397	validation_1-auc:0.659452
[3]	validation_0-auc:0.67775	validation_1-auc:0.664249
[4]	validation_0-auc:0.681572	validation_1-auc:0.668493
[5]	validation_0-auc:0.684365	validation_1-auc:0.671244
[6]	validation_0-auc:0.687832	validation_1-auc:0.672056
[7]	validation_0-auc:0.690781	validation_1-auc:0.675669
[8]	validation_0-auc:0.69585	validation_1-auc:0.680846
[9]	validation_0-auc:0.695661	validation_1-auc:0.680584
[10]	validation_0-auc:0.698799	validation_1-auc:0.684215
[11]	validation_0-auc:0.701046	validation_1-auc:0.686848
[12]	validation_0-auc:0.702624	validation_1-auc:0.687406
[13]	validation_0-auc:0.704234	validation_1-auc:0.688504
[14]	validation_0-auc:0.706435	validation_1-auc:0.690495
[15

18.851604461669922

In [131]:
test_datefrom

datetime.datetime(2018, 1, 1, 0, 0)

In [132]:
test_dateto

datetime.datetime(2019, 1, 1, 0, 0)

In [129]:
def strategy(row):
    strategy_input = pd.DataFrame([row])
    strategy_input = strategy_input.drop(['date','close','adj_close', 'volume'], axis=1)
    decision = 'buy' if model_class.predict(strategy_input) else 'sell'
    return decision

bot.simulate_trading(strategy, test_data, multipl=1000, interest_percent=0.05)

buy: open: 108.95, cash=502795.525, diff=2.8499999999999943, interest=54.475
sell: open: 112.19, cash=502705.525, diff=0.09000000000000341
sell: open: 112.95, cash=503425.525, diff=-0.7199999999999989
sell: open: 112.68, cash=504485.525, diff=-1.0600000000000023
buy: open: 110.89, cash=503560.08, diff=-0.8700000000000045, interest=55.445
buy: open: 110.13, cash=503315.015, diff=-0.18999999999999773, interest=55.065
sell: open: 109.96, cash=503805.015, diff=-0.4899999999999949
buy: open: 110.25, cash=504489.89, diff=0.7399999999999949, interest=55.125
buy: open: 111.32, cash=505584.23000000004, diff=1.1500000000000057, interest=55.660000000000004
sell: open: 113.07, cash=507964.23, diff=-2.3799999999999955
buy: open: 111.2, cash=508678.63, diff=0.769999999999996, interest=55.6
buy: open: 111.74, cash=507302.76, diff=-1.3199999999999932, interest=55.870000000000005
buy: open: 110.97, cash=506867.275, diff=-0.37999999999999545, interest=55.485
buy: open: 110.27, cash=507642.14, diff=0.829

buy: open: 104.15, cash=566258.7875000001, diff=1.1799999999999926, interest=52.075
buy: open: 105.88, cash=564365.8475000001, diff=-1.8399999999999892, interest=52.94
buy: open: 104.7, cash=564953.4975000002, diff=0.6400000000000006, interest=52.35
buy: open: 104.5, cash=565181.2475000002, diff=0.28000000000000114, interest=52.25
buy: open: 105.01, cash=566138.7425000002, diff=1.009999999999991, interest=52.505
buy: open: 106.25, cash=565865.6175000002, diff=-0.21999999999999886, interest=53.125
buy: open: 105.32, cash=568532.9575000003, diff=2.720000000000013, interest=52.660000000000004
buy: open: 108.05, cash=568678.9325000002, diff=0.20000000000000284, interest=54.025
buy: open: 108.0, cash=570624.9325000002, diff=2.0, interest=54.0
buy: open: 109.82, cash=570950.0225000002, diff=0.38000000000000966, interest=54.910000000000004
buy: open: 110.26, cash=570934.8925000002, diff=0.03999999999999204, interest=55.13
buy: open: 110.31, cash=571259.7375000002, diff=0.37999999999999545, in

sell: open: 99.3, cash=602416.6725000001, diff=-0.7099999999999937
buy: open: 98.85, cash=602967.2475, diff=0.6000000000000085, interest=49.425000000000004
buy: open: 99.54, cash=602917.4775, diff=0.0, interest=49.77
buy: open: 99.64, cash=603357.6575, diff=0.4899999999999949, interest=49.82
buy: open: 100.25, cash=604667.5325, diff=1.3599999999999994, interest=50.125
sell: open: 102.03, cash=606307.5325, diff=-1.6400000000000006
buy: open: 99.75, cash=606177.6575, diff=-0.0799999999999983, interest=49.875
buy: open: 99.7, cash=606447.8075, diff=0.3199999999999932, interest=49.85
buy: open: 100.39, cash=606877.6125, diff=0.480000000000004, interest=50.195
buy: open: 101.34, cash=606176.9425000001, diff=-0.6500000000000057, interest=50.67
buy: open: 101.15, cash=607676.3675000002, diff=1.5499999999999972, interest=50.575
sell: open: 104.74, cash=608116.3675000002, diff=-0.4399999999999977
sell: open: 104.64, cash=608166.3675000002, diff=-0.04999999999999716
buy: open: 104.5, cash=609064

buy: open: 83.4, cash=643192.7375000003, diff=0.8099999999999881, interest=41.7
buy: open: 84.46, cash=643510.5075000003, diff=0.35999999999999943, interest=42.230000000000004
buy: open: 84.59, cash=645348.2125000003, diff=1.8799999999999955, interest=42.295
buy: open: 86.53, cash=644754.9475000002, diff=-0.5499999999999972, interest=43.265
buy: open: 85.9, cash=645701.9975000003, diff=0.9899999999999949, interest=42.95
buy: open: 86.85, cash=644668.5725000002, diff=-0.9899999999999949, interest=43.425000000000004
buy: open: 85.9, cash=644375.6225000003, diff=-0.25, interest=42.95
sell: open: 85.65, cash=646025.6225000003, diff=-1.6500000000000057
buy: open: 84.09, cash=646333.5775000002, diff=0.3499999999999943, interest=42.045
buy: open: 84.94, cash=645921.1075000003, diff=-0.37000000000000455, interest=42.47
buy: open: 84.2, cash=646189.0075000003, diff=0.3100000000000023, interest=42.1
buy: open: 85.0, cash=647076.5075000003, diff=0.9300000000000068, interest=42.5
buy: open: 85.93,

buy: open: 92.05, cash=686216.5725000005, diff=-0.20000000000000284, interest=46.025
sell: open: 91.22, cash=686666.5725000005, diff=-0.45000000000000284
sell: open: 90.8, cash=686386.5725000005, diff=0.28000000000000114
buy: open: 91.29, cash=685600.9275000005, diff=-0.7400000000000091, interest=45.645
sell: open: 90.11, cash=688430.9275000005, diff=-2.8299999999999983
sell: open: 86.87, cash=688170.9275000005, diff=0.2599999999999909
sell: open: 86.54, cash=688890.9275000005, diff=-0.7200000000000131
buy: open: 86.42, cash=692837.7175000005, diff=3.989999999999995, interest=43.21
buy: open: 89.75, cash=694632.8425000005, diff=1.8400000000000034, interest=44.875
sell: open: 92.07, cash=694572.8425000005, diff=0.060000000000002274
sell: open: 92.67, cash=694092.8425000005, diff=0.480000000000004
buy: open: 91.64, cash=695747.0225000005, diff=1.7000000000000028, interest=45.82
sell: open: 93.21, cash=696097.0225000005, diff=-0.3499999999999943
sell: open: 93.21, cash=695867.0225000005, 

...

sell: open: 152.5, cash=4513449.25134999, diff=1.3000000000000114
sell: open: 152.88, cash=4514039.25134999, diff=-0.5900000000000034
sell: open: 150.85, cash=4514889.25134999, diff=-0.8499999999999943
buy: open: 150.27, cash=4517614.1163499905, diff=2.799999999999983, interest=75.135
sell: open: 153.08, cash=4515754.1163499905, diff=1.8599999999999852
sell: open: 154.2, cash=4514094.1163499905, diff=1.660000000000025
sell: open: 157.5, cash=4514774.1163499905, diff=-0.6800000000000068
sell: open: 156.41, cash=4517884.1163499905, diff=-3.109999999999985
sell: open: 154.15, cash=4518114.1163499905, diff=-0.2300000000000182
sell: open: 154.11, cash=4519524.1163499905, diff=-1.410000000000025
sell: open: 155.48, cash=4517244.1163499905, diff=2.280000000000001
sell: open: 155.79, cash=4516734.1163499905, diff=0.5100000000000193
sell: open: 156.25, cash=4518304.1163499905, diff=-1.5699999999999932
buy: open: 163.25, cash=4520222.4913499905, diff=2.0, interest=81.625
sell: open: 166.11, cash

buy: open: 101.49, cash=4574245.98634999, diff=-0.06999999999999318, interest=50.745000000000005
buy: open: 101.69, cash=4573795.14134999, diff=-0.3999999999999915, interest=50.845
buy: open: 100.82, cash=4573664.73134999, diff=-0.0799999999999983, interest=50.410000000000004
buy: open: 100.28, cash=4573924.59134999, diff=0.3100000000000023, interest=50.14
buy: open: 100.65, cash=4574444.26634999, diff=0.5699999999999932, interest=50.325
buy: open: 100.93, cash=4574453.80134999, diff=0.05999999999998806, interest=50.465
buy: open: 101.57, cash=4574873.01634999, diff=0.4700000000000131, interest=50.785000000000004
buy: open: 102.32, cash=4575201.85634999, diff=0.38000000000000966, interest=51.160000000000004
buy: open: 101.82, cash=4575260.94634999, diff=0.11000000000001364, interest=50.910000000000004
buy: open: 101.69, cash=4575530.10134999, diff=0.3200000000000074, interest=50.845
buy: open: 101.35, cash=4575229.42634999, diff=-0.25, interest=50.675000000000004
sell: open: 99.84, cas

buy: open: 106.65, cash=4644622.381349986, diff=1.2099999999999937, interest=53.325
sell: open: 108.61, cash=4645462.381349986, diff=-0.8400000000000034
buy: open: 106.68, cash=4648329.041349987, diff=2.9199999999999875, interest=53.34
sell: open: 108.58, cash=4647449.041349987, diff=0.8799999999999955
sell: open: 109.9, cash=4649099.041349987, diff=-1.6500000000000057
sell: open: 107.55, cash=4650559.041349987, diff=-1.4599999999999937
sell: open: 106.37, cash=4650429.041349987, diff=0.12999999999999545
sell: open: 105.88, cash=4650569.041349987, diff=-0.14000000000000057
buy: open: 107.15, cash=4653045.466349986, diff=2.530000000000001, interest=53.575
sell: open: 109.5, cash=4652435.466349986, diff=0.6099999999999994
buy: open: 110.33, cash=4654940.301349986, diff=2.5600000000000023, interest=55.165
buy: open: 112.0, cash=4654194.301349986, diff=-0.6899999999999977, interest=56.0
buy: open: 110.37, cash=4656039.116349987, diff=1.8999999999999915, interest=55.185
buy: open: 113.99, c

sell: open: 232.12, cash=4743117.736349987, diff=0.3299999999999841
buy: open: 233.0, cash=4744141.236349987, diff=1.1399999999999864, interest=116.5
sell: open: 232.04, cash=4753011.236349987, diff=-8.870000000000005
sell: open: 223.5, cash=4761151.236349987, diff=-8.139999999999986
sell: open: 217.86, cash=4758771.236349987, diff=2.3799999999999955
sell: open: 221.02, cash=4763921.236349987, diff=-5.150000000000006
buy: open: 217.23, cash=4763122.621349987, diff=-0.6899999999999977, interest=108.61500000000001
buy: open: 217.79, cash=4764743.726349987, diff=1.7300000000000182, interest=108.895
sell: open: 218.05, cash=4770173.726349987, diff=-5.430000000000007
sell: open: 213.0, cash=4766843.726349987, diff=3.3300000000000125
buy: open: 212.39, cash=4771907.531349987, diff=5.170000000000016, interest=106.19500000000001
sell: open: 218.64, cash=4771877.531349987, diff=0.030000000000001137
sell: open: 217.4, cash=4777027.531349987, diff=-5.150000000000006
sell: open: 213.75, cash=47779

buy: open: 210.24, cash=4860232.306349986, diff=1.289999999999992, interest=105.12
buy: open: 211.77, cash=4861476.421349986, diff=1.3499999999999943, interest=105.885
buy: open: 214.7, cash=4861939.071349987, diff=0.5700000000000216, interest=107.35000000000001
buy: open: 217.32, cash=4860840.411349987, diff=-0.9899999999999807, interest=108.66
buy: open: 215.39, cash=4858852.716349986, diff=-1.8799999999999955, interest=107.69500000000001
sell: open: 214.13, cash=4862342.716349986, diff=-3.490000000000009
buy: open: 211.49, cash=4861816.971349986, diff=-0.4200000000000159, interest=105.745
buy: open: 212.0, cash=4859820.971349986, diff=-1.8899999999999864, interest=106.0
buy: open: 209.53, cash=4860896.206349987, diff=1.1800000000000068, interest=104.765
buy: open: 212.4, cash=4860580.006349986, diff=-0.21000000000000796, interest=106.2
buy: open: 212.38, cash=4863803.816349986, diff=3.3300000000000125, interest=106.19
buy: open: 216.0, cash=4863455.816349986, diff=-0.240000000000009